In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sqlalchemy.engine import create_engine
from mysql.connector import connect
import mysql.connector
from mysql.connector import Error
import pymysql
import warnings
import traceback
from pandas.io import sql
from sqlalchemy import create_engine

from random import randrange
import datetime
from datetime import timedelta
from sklearn import preprocessing
# import datetime

# import MySQLdb

warnings.filterwarnings("ignore")

In [2]:
from urllib.parse import quote_plus


In [3]:
%load_ext nb_black


<IPython.core.display.Javascript object>

In [4]:
%load_ext sql

<IPython.core.display.Javascript object>

In [5]:
# user_name = "root"
# pwd = "Trung99"
# server = "localhost"
# db = "tastie"
# # conn = connect(host=server, user=user_name, password=pwd, database=db, port=3306)
# # cur = conn.cursor(buffered=True)

<IPython.core.display.Javascript object>

In [6]:
# cur.close()
# conn.close()

<IPython.core.display.Javascript object>

In [7]:
# def insert_varibles_into_table(lst_value):
#     try:
#         connection = mysql.connector.connect(host='localhost',
#                                              database='Electronics',
#                                              user='pynative',
#                                              password='pynative@#29')
#         cursor = connection.cursor()
#         for i in lst_value:
#         mySql_insert_query = """INSERT INTO Laptop (Id, Name, Price, Purchase_date) 
#                                 VALUES (%s, %s, %s, %s) """

#         record = (id, name, price, purchase_date)
#         cursor.execute(mySql_insert_query, record)
#         connection.commit()
#         print("Record inserted successfully into Laptop table")

#     except mysql.connector.Error as error:
#         print("Failed to insert into MySQL table {}".format(error))

#     finally:
#         if connection.is_connected():
#             cursor.close()
#             connection.close()
#             print("MySQL connection is closed")


<IPython.core.display.Javascript object>

In [8]:
# connect db
def connect_db():
    user_name = "root"
    pwd = "Trung99"
    server = "localhost"
    db = "tastie"
    conn = connect(host=server, user=user_name, password=pwd, database=db, port=3306)
    cur = conn.cursor(buffered=True)
    return conn, cur


# conn = create_engine(conn_str)

<IPython.core.display.Javascript object>

In [9]:
def close_connection(cur, conn):
    cur.close()
    conn.close()

<IPython.core.display.Javascript object>

In [10]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


<IPython.core.display.Javascript object>

In [11]:
def insert_execute_many(dataset, table_name):
    conn, cur = connect_db()

    cols = "`,`".join([str(i) for i in list(dataset)])
    sql = (
        "INSERT INTO `"
        + table_name
        + "` (`"
        + cols
        + "`) VALUES ("
        + "%s," * (dataset.shape[1] - 1)
        + "%s)"
    )

    try:
        cur.executemany(sql, dataset.values.tolist())
        conn.commit()
    except Error as error:
        conn.rollback()
        print("Failed to insert into MySQL table {}".format(error))
    finally:
        cur.close()
        conn.close()

<IPython.core.display.Javascript object>

In [12]:
# for dataset > 100.000 records and many columns (ex: product)
def insert_execute_many_large_dataset(dataset, table_name):
    conn, cur = connect_db()
    dataset_size = len(dataset)
    step = 100000

    for start in range(0, len(dataset), step):
        if dataset_size < step:
            step = abs(step-dataset_size)
            end = start + step
            insert_execute_many(dataset[start:end], table_name)
            break
        end = start + step
        insert_execute_many(dataset[start:end], table_name) 

        dataset_size -= step


<IPython.core.display.Javascript object>

### User

In [13]:
df_user = pd.read_csv("data/user_info.csv")

<IPython.core.display.Javascript object>

In [14]:
# df_user = pd.read_sql_query("SELECT * FROM user", conn)

<IPython.core.display.Javascript object>

In [15]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39128 entries, 0 to 39127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        39128 non-null  int64 
 1   first_name     39128 non-null  object
 2   last_name      39128 non-null  object
 3   gender         39128 non-null  int64 
 4   birthday       39128 non-null  object
 5   email          39128 non-null  object
 6   phone          39128 non-null  int64 
 7   password       39128 non-null  object
 8   role           39128 non-null  int64 
 9   user_token     39128 non-null  object
 10  registered_at  39128 non-null  object
 11  update_at      39128 non-null  object
 12  delete_at      39128 non-null  object
dtypes: int64(4), object(9)
memory usage: 3.9+ MB


<IPython.core.display.Javascript object>

In [16]:
df_user

,user_id,first_name,last_name,gender,birthday,email,phone,password,role,user_token,registered_at,update_at,delete_at
0,1000001,Adaline,Rolofs,0,4/11/1986,Altstadt@hubspot.com,127223056,i87Z8UWYV0x,3,6Sd5oV14jY=4hGnBF19T698P45MJilxG#9I49MbgL8txjCU1,12/11/2019 19:47,4/25/2005 8:19,1/1/1971 0:16
1,1000002,Lenny,Marquez,1,1/6/1970,Bounds@hubspot.com,814852386,zZS260t3a6oF&64wUJ855qLTe03819UpGTfM+Q7m,2,7VLM5aZU44H9#9eKRdf36985H9&V643g=1Ip5I62#D,1/1/2018 0:00,12/25/2013 13:48,7/8/2010 9:00
2,1000003,Camara,Zywiyask,0,8/17/1988,Deimante.Leyendecker64769@yahoo.com,553575365,y5Y1EuCTa,3,p3hs5SfM2P35N1cPyoWT%3+gH5Gosczz4Cf046Ey0tO55N...,1/1/2018 1:53,8/22/1995 4:43,1/1/1971 0:00
3,1000004,Seni,Besga,0,1/12/1975,MistiAThamry1672431299849@gmail.com,554173329,rxK%0v#1v178qsyzVNFRa5C2ar849MmPkBq4vu0FiSlU3u96,2,M2pJzT2+9758fj2twtO4s5x9p3j4N12K07mN3gZgSYH2j7...,2/7/2019 13:30,10/29/1980 22:13,1/1/1971 0:02
4,1000005,Jaunita,Abdurakhmanoff,1,1/7/1975,AkilahOrta6@hubspot.com,980690649,QYG44f^nV67,1,4k97r92q270T9Pzkc0Fc4Il^MBb1MSC9,1/9/2022 5:12,12/8/2002 22:43,3/21/2015 19:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39123,1039124,Daryl,Hollerbaum,1,9/30/1983,DebbieAlonzo@gmail.com,982536330,D5z9i12%g7uB87tl,3,W841TS^K8dyEq0A4TIOL0oaYZC1IpGFS#,11/27/2018 22:23,1/1/1971 0:00,1/1/1971 0:11
39124,1039125,Romario,Groenen,1,6/10/1999,AdenaKreuzeder76@hubspot.com,883815268,+8D1h0nEvmX7440kC2d9153&,3,byZvU54Y5p2=C=SL2uCzpllRm0PYGBOkKUnzB133Y673xG...,8/4/2019 13:48,1/1/1971 0:01,4/23/2004 4:32
39125,1039126,Adelia,Larburu,1,10/18/1998,Masegosa@icloud.com,815569692,&&la7Jys$2,3,0Z2hwq2yoev&QM85i34opVql4n%4V0zZ=4A3Ron586gMab...,11/24/2021 12:49,1/1/1971 0:00,1/1/1971 0:01
39126,1039127,Lluisa,Hollerbuhl,0,8/17/1987,NovellaAmaya@hubspot.com,630507932,UuH39zB33%Q3L7Np9,1,unDZ3to19u08,4/14/2018 1:59,4/3/1973 9:34,1/1/1971 0:02


<IPython.core.display.Javascript object>

In [17]:
df_user["phone"] = df_user["phone"].apply(lambda x: "0" + str(x))

<IPython.core.display.Javascript object>

In [18]:
df_user["registered_at"] = pd.to_datetime(df_user["registered_at"])
df_user["update_at"] = pd.to_datetime(df_user["registered_at"])  # trick nha
df_user["delete_at"] = pd.to_datetime(df_user["delete_at"])
df_user["birthday"] = pd.to_datetime(df_user["birthday"])


<IPython.core.display.Javascript object>

In [19]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39128 entries, 0 to 39127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        39128 non-null  int64         
 1   first_name     39128 non-null  object        
 2   last_name      39128 non-null  object        
 3   gender         39128 non-null  int64         
 4   birthday       39128 non-null  datetime64[ns]
 5   email          39128 non-null  object        
 6   phone          39128 non-null  object        
 7   password       39128 non-null  object        
 8   role           39128 non-null  int64         
 9   user_token     39128 non-null  object        
 10  registered_at  39128 non-null  datetime64[ns]
 11  update_at      39128 non-null  datetime64[ns]
 12  delete_at      39128 non-null  datetime64[ns]
dtypes: datetime64[ns](4), int64(3), object(6)
memory usage: 3.9+ MB


<IPython.core.display.Javascript object>

In [20]:
df_user.head()

,user_id,first_name,last_name,gender,birthday,email,phone,password,role,user_token,registered_at,update_at,delete_at
0,1000001,Adaline,Rolofs,0,1986-04-11,Altstadt@hubspot.com,0127223056,i87Z8UWYV0x,3,6Sd5oV14jY=4hGnBF19T698P45MJilxG#9I49MbgL8txjCU1,2019-12-11 19:47:00,2019-12-11 19:47:00,1971-01-01 00:16:00
1,1000002,Lenny,Marquez,1,1970-01-06,Bounds@hubspot.com,0814852386,zZS260t3a6oF&64wUJ855qLTe03819UpGTfM+Q7m,2,7VLM5aZU44H9#9eKRdf36985H9&V643g=1Ip5I62#D,2018-01-01 00:00:00,2018-01-01 00:00:00,2010-07-08 09:00:00
2,1000003,Camara,Zywiyask,0,1988-08-17,Deimante.Leyendecker64769@yahoo.com,0553575365,y5Y1EuCTa,3,p3hs5SfM2P35N1cPyoWT%3+gH5Gosczz4Cf046Ey0tO55N...,2018-01-01 01:53:00,2018-01-01 01:53:00,1971-01-01 00:00:00
3,1000004,Seni,Besga,0,1975-01-12,MistiAThamry1672431299849@gmail.com,0554173329,rxK%0v#1v178qsyzVNFRa5C2ar849MmPkBq4vu0FiSlU3u96,2,M2pJzT2+9758fj2twtO4s5x9p3j4N12K07mN3gZgSYH2j7...,2019-02-07 13:30:00,2019-02-07 13:30:00,1971-01-01 00:02:00
4,1000005,Jaunita,Abdurakhmanoff,1,1975-01-07,AkilahOrta6@hubspot.com,0980690649,QYG44f^nV67,1,4k97r92q270T9Pzkc0Fc4Il^MBb1MSC9,2022-01-09 05:12:00,2022-01-09 05:12:00,2015-03-21 19:33:00


<IPython.core.display.Javascript object>

In [21]:
df_user.delete_at = None
df_user["avatar"] = None

<IPython.core.display.Javascript object>

In [22]:
df_user.head()

,user_id,first_name,last_name,gender,birthday,email,phone,password,role,user_token,registered_at,update_at,delete_at,avatar
0,1000001,Adaline,Rolofs,0,1986-04-11,Altstadt@hubspot.com,0127223056,i87Z8UWYV0x,3,6Sd5oV14jY=4hGnBF19T698P45MJilxG#9I49MbgL8txjCU1,2019-12-11 19:47:00,2019-12-11 19:47:00,None,None
1,1000002,Lenny,Marquez,1,1970-01-06,Bounds@hubspot.com,0814852386,zZS260t3a6oF&64wUJ855qLTe03819UpGTfM+Q7m,2,7VLM5aZU44H9#9eKRdf36985H9&V643g=1Ip5I62#D,2018-01-01 00:00:00,2018-01-01 00:00:00,None,None
2,1000003,Camara,Zywiyask,0,1988-08-17,Deimante.Leyendecker64769@yahoo.com,0553575365,y5Y1EuCTa,3,p3hs5SfM2P35N1cPyoWT%3+gH5Gosczz4Cf046Ey0tO55N...,2018-01-01 01:53:00,2018-01-01 01:53:00,None,None
3,1000004,Seni,Besga,0,1975-01-12,MistiAThamry1672431299849@gmail.com,0554173329,rxK%0v#1v178qsyzVNFRa5C2ar849MmPkBq4vu0FiSlU3u96,2,M2pJzT2+9758fj2twtO4s5x9p3j4N12K07mN3gZgSYH2j7...,2019-02-07 13:30:00,2019-02-07 13:30:00,None,None
4,1000005,Jaunita,Abdurakhmanoff,1,1975-01-07,AkilahOrta6@hubspot.com,0980690649,QYG44f^nV67,1,4k97r92q270T9Pzkc0Fc4Il^MBb1MSC9,2022-01-09 05:12:00,2022-01-09 05:12:00,None,None


<IPython.core.display.Javascript object>

In [23]:
df_user.to_csv("mysql_data/user.csv", index=False)

<IPython.core.display.Javascript object>

In [24]:
# conn, cur = connect_db()
# cols = "`,`".join([str(i) for i in list(df_user.columns)])

# for i, row in df_user.iterrows():
#     sql = (
#         "INSERT INTO `user` (`" + cols + "`) VALUES (" + "%s," * (len(row) - 1) + "%s)"
#     )
#     #     print(sql)
#     #     print(tuple(row))
#     #     break

#     cur.execute(sql, tuple(row))

# conn.commit()
# cur.close()
# conn.close()

<IPython.core.display.Javascript object>

### Cart

In [25]:
# df_user[df_user.role == 1][["user_id"]]

<IPython.core.display.Javascript object>

In [26]:
df_cart = df_user[df_user.role == 1][["user_id"]]
df_cart.rename(columns={"user_id": "cart_id"}, inplace=True)

<IPython.core.display.Javascript object>

In [27]:
df_cart.head()

,cart_id
4,1000005
5,1000006
7,1000008
11,1000012
13,1000014


<IPython.core.display.Javascript object>

In [28]:
df_cart["user_id"] = df_cart["cart_id"]

<IPython.core.display.Javascript object>

In [29]:
df_cart["cart_state"] = 1
df_cart["total_price"] = 0

<IPython.core.display.Javascript object>

In [30]:
df_cart

,cart_id,user_id,cart_state,total_price
4,1000005,1000005,1,0
5,1000006,1000006,1,0
7,1000008,1000008,1,0
11,1000012,1000012,1,0
13,1000014,1000014,1,0
...,...,...,...,...
39118,1039119,1039119,1,0
39119,1039120,1039120,1,0
39120,1039121,1039121,1,0
39126,1039127,1039127,1,0


<IPython.core.display.Javascript object>

In [31]:
cart_values = df_cart.values.tolist()

<IPython.core.display.Javascript object>

In [32]:
cart_values[:2]

[[1000005, 1000005, 1, 0], [1000006, 1000006, 1, 0]]

<IPython.core.display.Javascript object>

In [33]:
# conn, cur = connect_db()
# cols = "`,`".join([str(i) for i in list(df_cart.columns)])
# sql = 'INSERT INTO Cart (cart_id, user_id, cart_state, total_price) VALUES (%s, %s, %s, %s)'
# cur.executemany(sql, cart_values)
# conn.commit()
# cur.close()
# conn.close()


<IPython.core.display.Javascript object>

In [34]:
df_cart.to_csv("mysql_data/cart.csv", index=False)

<IPython.core.display.Javascript object>

In [35]:
# insert_execute_many(df_cart, "Cart")

<IPython.core.display.Javascript object>

### Owner

In [36]:
df_owner = pd.read_csv("data/owner_info.csv")

<IPython.core.display.Javascript object>

In [37]:
df_owner

,owner_id,owner_card_id,owner_phone,owner_name,email,company_name,company_address,role,create_at,update_at,delete_at,bank_id_card_number,date_of_issue,bank_beneficiary_name,bank_account_number,bank_name,bank_province,bank_branch
0,1000000,804601922286,459861314,Lashaunda241,Thielsch@zoho.com,WorldWide Optics Corporation,"461 NW Riddle Hill Lane, Concord, NH, 04357",1,1/1/2018 12:07:13 AM,1/1/2018 12:00:04 AM,12/26/1974 4:36:38 AM,5553325242948090,3/8/2024,GAAA HR CS S5E,9463011450082423,Pacific Renewable Energy Corporation,240 Hidden Front Way,Zuni
1,1000001,849082762300,242675663,Liubov1975,Ames974385857@gmail.com,Advanced Research Corp.,"11 Red Lake Hwy, Harrisburg, Pennsylvania, 30254",1,10/19/2019 1:01:51 PM,5/18/2021 12:49:32 PM,8/11/2021 9:53:46 PM,4844029508634350,7/21/2023,AYNU CA 76 48Q,4958767034927343,United Z-Mobile Corp.,2948 Stonewood Pkwy,Solon Springs
2,1000002,368767478644,567642224,Jocelyn5,Morris_Gensrich16241444@gmail.com,National Space Explore Co.,"1753 North Church Pkwy, Boise, Idaho, 95705",1,9/25/2020 4:52:24 AM,1/1/2018 12:00:08 AM,1/1/1971 1:19:23 AM,30222044191520,8/1/2026,SHQX NZ PE A85,7357757843438961,Advanced Space Research Inc.,44 West Church Ct,Waconia
3,1000003,496315878400,441915299,Thiebault1974,GyulaBurnside@yahoo.com,Smart Mining Corporation,"1695 Hidden Deepwood Cir, Superior Bldg, India...",1,10/31/2020 7:22:48 PM,5/19/2020 4:23:22 PM,2/2/1993 10:40:48 AM,6280874217787445,8/14/2026,RDNU CO 81 T62,3999399748405384,General Petroleum Inc.,3778 Edgewood Highway,Solvang
4,1000004,937535134115,986434754,Katherine368,HesseniusL57@gmail.com,Global High-Technologies Corp.,"970 Riddle Hill Pkwy, Comcast Building, Albany...",1,10/26/2020 9:11:48 AM,6/9/2018 7:31:29 AM,6/18/1981 4:34:02 AM,5472617632079249,1/26/2023,UOUL IT L2 P27,2788798796848619,City Space Explore Group,12 1st Circle,Lonsdale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39865,1039865,616834345606,103275255,Lawanna719,Pinter@gmail.com,Professional Wave Energy Corporation,"850 E Ashwood Loop, Equitable Building, Richmo...",1,1/16/2018 9:24:04 AM,5/20/2018 10:50:32 AM,5/6/1980 9:50:02 PM,3587304919923897,11/29/2024,YDHI JP 2L,7371321065874441,American Business Corporation,51 Ironwood Ln,Udall
39866,1039866,152334225117,395361722,Abbott74,Doroteo_Adamson11@icloud.com,United Telecommunications Corp.,"263 Mount Hwy, Bismarck, North Dakota, 75644",1,9/23/2020 9:20:50 PM,1/27/2022 5:17:36 PM,1/1/1971 12:38:19 AM,30091443416337,2/18/2023,OAYY AF V3,3048145587478905,First 3G Wireless Inc.,80 Prospect Hill Ln,Mott
39867,1039867,130824450300,494237072,Bonny1996,Roca.Hyde@icloud.com,First 3G Wireless Group,"50 Stonewood Highway, Albany, NY, 79050",2,1/1/2018 12:00:07 AM,12/10/2020 12:21:27 AM,9/1/1996 5:45:05 AM,343087895734427,4/28/2023,LCBP RO RL,4661771645630129,Canadian K-Mobile Corporation,1701 Prospect Hill Circle,Lemont Furnace
39868,1039868,484570745577,203181142,Brigid1964,AdahNigmann9363@yahoo.com,Canadian High-Technologies Inc.,"3032 West Social St, Springfield, Illinois, 48002",2,4/22/2021 3:52:51 PM,12/29/2020 4:48:43 AM,1/1/1971 12:13:18 AM,5245997038363330,5/23/2023,WXRP IT IT JQ2,2785797782395172,Australian Nuclear Power Group,3201 NE Chapel Hill Loop,Moulton


<IPython.core.display.Javascript object>

In [38]:
df_owner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39870 entries, 0 to 39869
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   owner_id               39870 non-null  int64 
 1   owner_card_id          39870 non-null  int64 
 2   owner_phone            39870 non-null  int64 
 3   owner_name             39870 non-null  object
 4   email                  39870 non-null  object
 5   company_name           39870 non-null  object
 6   company_address        39870 non-null  object
 7   role                   39870 non-null  int64 
 8   create_at              39870 non-null  object
 9   update_at              39870 non-null  object
 10  delete_at              39870 non-null  object
 11  bank_id_card_number    39870 non-null  int64 
 12  date_of_issue          39870 non-null  object
 13  bank_beneficiary_name  39870 non-null  object
 14  bank_account_number    39870 non-null  int64 
 15  bank_name          

<IPython.core.display.Javascript object>

In [39]:
df_owner["create_at"] = pd.to_datetime(df_owner["create_at"])
df_owner["update_at"] = df_owner["create_at"]
df_owner["date_of_issue"] = pd.to_datetime(df_owner["date_of_issue"])

<IPython.core.display.Javascript object>

In [40]:
# df_owner.role = df_owner.role.astype("str")
# df_owner.bank_id_card_number = df_owner.bank_id_card_number.astype("str")
# df_owner.bank_account_number = df_owner.bank_account_number.astype("str")
# df_owner.owner_phone = df_owner.owner_phone.astype("str")
# df_owner.owner_card_id = df_owner.owner_card_id.astype("str")
# df_owner.owner_id = df_owner.owner_id.astype("str")

<IPython.core.display.Javascript object>

In [41]:
lst_bank_name = [
    "JPMorgan Chase & Co.",
    "Bank of America Corp.",
    "Wells Fargo & Co.",
    "Citigroup Inc.",
    "U.S. Bancorp",
    "Truist Bank",
    "PNC Financial Services Group Inc.",
    "TD Group US Holdings LLC",
    "Goldman Sachs Group Inc.",
    "Bank of New York Mellon Corp.",
    "Capital One Financial Corp.",
    "State Street Corp.",
    "Fifth Third Bank",
    "HSBC",
    "Silicon Valley Bank",
]

<IPython.core.display.Javascript object>

In [42]:
df_owner.bank_name = df_owner.bank_name.apply(lambda x: random.choice(lst_bank_name))

<IPython.core.display.Javascript object>

In [43]:
df_owner.bank_name.unique()

array(['HSBC', 'Citigroup Inc.', 'Bank of New York Mellon Corp.',
       'Capital One Financial Corp.', 'TD Group US Holdings LLC',
       'PNC Financial Services Group Inc.', 'Silicon Valley Bank',
       'JPMorgan Chase & Co.', 'Fifth Third Bank', 'U.S. Bancorp',
       'State Street Corp.', 'Truist Bank', 'Wells Fargo & Co.',
       'Bank of America Corp.', 'Goldman Sachs Group Inc.'], dtype=object)

<IPython.core.display.Javascript object>

In [44]:
df_owner.delete_at = None

<IPython.core.display.Javascript object>

In [45]:
df_user[df_user.role == 2].loc[:, ["user_id", "registered_at", "update_at"]].shape

(12927, 3)

<IPython.core.display.Javascript object>

In [46]:
df_owner.shape

(39870, 18)

<IPython.core.display.Javascript object>

In [47]:
df_owner = df_owner.iloc[:12927, :]

<IPython.core.display.Javascript object>

In [48]:
owner_values = df_owner.values.tolist()

<IPython.core.display.Javascript object>

In [49]:
# owner_values[:2]

<IPython.core.display.Javascript object>

In [50]:
df_owner.columns

Index(['owner_id', 'owner_card_id', 'owner_phone', 'owner_name', 'email',
       'company_name', 'company_address', 'role', 'create_at', 'update_at',
       'delete_at', 'bank_id_card_number', 'date_of_issue',
       'bank_beneficiary_name', 'bank_account_number', 'bank_name',
       'bank_province', 'bank_branch'],
      dtype='object')

<IPython.core.display.Javascript object>

In [51]:
# bị big lost connect server nên phải chia ra 2 khoảng rồi insert :(())

<IPython.core.display.Javascript object>

In [52]:
df_owner.to_csv("mysql_data/owner.csv", index=False)

<IPython.core.display.Javascript object>

In [53]:
# conn, cur = connect_db()
# cols = "`,`".join([str(i) for i in list(df_owner.columns)])
# sql = (
#     "INSERT INTO `owner` (`"
#     + cols
#     + "`) VALUES ("
#     + "%s," * (df_owner.shape[1] - 1)
#     + "%s)"
# )

# try:
#     cur.executemany(sql, owner_values[:6000])  # owner_values
#     conn.commit()
# except Error as error:
#     # conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
# finally:
#     cur.close()
#     conn.close()

<IPython.core.display.Javascript object>

In [54]:
# insert_execute_many(df_owner, "Owner")

<IPython.core.display.Javascript object>

### provider 

In [55]:
df = pd.read_csv("data/output_restaurant.csv")


<IPython.core.display.Javascript object>

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511450 entries, 0 to 511449
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   restaurant_description  290189 non-null  object 
 1   rating                  511450 non-null  float64
 2   num_review              511450 non-null  int64  
 3   menu_category           509449 non-null  object 
 4   product_name            509449 non-null  object 
 5   price                   504983 non-null  float64
 6   product_description     286605 non-null  object 
 7   status                  482712 non-null  object 
 8   product_image           253880 non-null  object 
 9   restaurant_url          511450 non-null  object 
 10  restaurant_name         511450 non-null  object 
 11  avt_restaurant          511202 non-null  object 
 12  restaurant_category     511450 non-null  object 
 13  city                    511450 non-null  object 
dtypes: float64(2), int64

<IPython.core.display.Javascript object>

In [57]:
df.restaurant_category = df.restaurant_category.fillna("others")
df.menu_category = df.menu_category.fillna("others")
df = df[df.product_name.notna()]
df = df[df.price.notna()]

df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [58]:
le = preprocessing.LabelEncoder()
df["product_id"] = le.fit_transform(df["product_name"])

df["product_id"] = df["product_id"].apply(lambda x: x + 1000000)

<IPython.core.display.Javascript object>

In [59]:
# # restaurant_url_group = df.groupby("restaurant_url")

# id_range = list(range(1000000, 1000000 + len(df), 1))
# df["provider_id"] = df[["restaurant_url"]].sum(axis=1).map(hash)

# df["provider_id"] = df.groupby("restaurant_url").ngroup()
# df["provider_id"] = df["provider_id"] + 1000000

<IPython.core.display.Javascript object>

In [59]:
le = preprocessing.LabelEncoder()
df["provider_id"] = le.fit_transform(df["restaurant_url"])

df["provider_id"] = df["provider_id"].apply(lambda x: x + 1000000)

<IPython.core.display.Javascript object>

In [61]:
# df[df.price.isna()].to_csv("test.csv", index=False)

<IPython.core.display.Javascript object>

In [62]:
# df.price = df.price.interpolate()

<IPython.core.display.Javascript object>

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504914 entries, 0 to 504913
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   restaurant_description  286851 non-null  object 
 1   rating                  504914 non-null  float64
 2   num_review              504914 non-null  int64  
 3   menu_category           504914 non-null  object 
 4   product_name            504914 non-null  object 
 5   price                   504914 non-null  float64
 6   product_description     282070 non-null  object 
 7   status                  476176 non-null  object 
 8   product_image           250999 non-null  object 
 9   restaurant_url          504914 non-null  object 
 10  restaurant_name         504914 non-null  object 
 11  avt_restaurant          504666 non-null  object 
 12  restaurant_category     504914 non-null  object 
 13  city                    504914 non-null  object 
 14  product_id          

<IPython.core.display.Javascript object>

In [61]:
# df["Q1_price"] = df.groupby("provider_id").price.quantile(0.25)
# df["Q3_price"] = df.groupby("provider_id").price.quantile(0.75)

<IPython.core.display.Javascript object>

In [62]:
df_test = df.merge(
    df.groupby("provider_id").price.quantile(0.25).to_frame(), on="provider_id"
).copy()

<IPython.core.display.Javascript object>

In [63]:
df_test.head(1)

,restaurant_description,rating,num_review,menu_category,product_name,price_x,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id,price_y
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002540,1000047,1.59


<IPython.core.display.Javascript object>

In [64]:
df_test = df_test.rename(columns={"price_x": "price", "price_y": "price_min"})

<IPython.core.display.Javascript object>

In [65]:
df_test = df_test.merge(
    df_test.groupby("provider_id").price.quantile(0.75).to_frame(), on="provider_id"
)

<IPython.core.display.Javascript object>

In [66]:
df_test = df_test.rename(columns={"price_x": "price", "price_y": "price_max"})

<IPython.core.display.Javascript object>

In [67]:
df = df_test.copy()

<IPython.core.display.Javascript object>

In [68]:
# df.product_image = df.product_image.apply(lambda x: str(x).replace("https://", ""))
# df.avt_restaurant = df.avt_restaurant.apply(lambda x: str(x).replace("https://", ""))

<IPython.core.display.Javascript object>

In [69]:
# df.head()

<IPython.core.display.Javascript object>

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504914 entries, 0 to 504913
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   restaurant_description  286851 non-null  object 
 1   rating                  504914 non-null  float64
 2   num_review              504914 non-null  int64  
 3   menu_category           504914 non-null  object 
 4   product_name            504914 non-null  object 
 5   price                   504914 non-null  float64
 6   product_description     282070 non-null  object 
 7   status                  476176 non-null  object 
 8   product_image           250999 non-null  object 
 9   restaurant_url          504914 non-null  object 
 10  restaurant_name         504914 non-null  object 
 11  avt_restaurant          504666 non-null  object 
 12  restaurant_category     504914 non-null  object 
 13  city                    504914 non-null  object 
 14  product_id          

<IPython.core.display.Javascript object>

In [71]:
df_provider = df[
    [
        "provider_id",
        "restaurant_url",
        "restaurant_name",
        "restaurant_category",
        "avt_restaurant",
        "restaurant_description",
        "price_min",
        "price_max",
    ]
]

<IPython.core.display.Javascript object>

In [72]:
df_provider.drop_duplicates(subset=["restaurant_url"], inplace=True)

<IPython.core.display.Javascript object>

In [73]:
df_provider.reset_index(inplace=True)

<IPython.core.display.Javascript object>

In [74]:
df_provider

,index,provider_id,restaurant_url,restaurant_name,restaurant_category,avt_restaurant,restaurant_description,price_min,price_max
0,0,1000047,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,NaN,1.59,3.69
1,4350,1000033,https://www.ubereats.com/ca/store/hambrgr-down...,HAMBRGR Downtown,24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,You've found one of the hottest delivery spots...,9.00,16.00
2,12306,1000061,https://www.ubereats.com/ca/store/popeyes-loui...,Popeyes Louisiana Kitchen (11 King St E),24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,NaN,3.79,14.49
3,24150,1000102,https://www.ubereats.com/ca/store/wendys-244-h...,Wendy's (244 Hwy 20 South),24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,Wish you could ask someone what's best here? O...,2.74,9.89
4,24223,1000094,https://www.ubereats.com/ca/store/the-ship/MHS...,The Ship,24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,NaN,9.00,18.00
...,...,...,...,...,...,...,...,...,...
102,474059,1000071,https://www.ubereats.com/ca/store/smokes-pouti...,Smoke's Poutinerie (Hamilton),bacon,https://cn-geo1.uber.com/image-proc/resize/eat...,Here's one of the hottest delivery spots in al...,12.99,19.99
103,475770,1000034,https://www.ubereats.com/ca/store/harveys-fenn...,Harvey's (Fennell Ave),appetizers,https://cn-geo1.uber.com/image-proc/resize/eat...,This place is one of the hottest places for de...,5.84,11.49
104,477954,1000079,https://www.ubereats.com/ca/store/sunrise-mark...,Sunrise Market (Cannon St),24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,Wish you could ask Sunrise Market customers wh...,3.99,8.99
105,494194,1000005,https://www.ubereats.com/ca/store/anytime-conv...,ANYTIME CONVENIENCE,24-hours,https://cn-geo1.uber.com/image-proc/resize/eat...,NaN,3.97,8.88


<IPython.core.display.Javascript object>

In [75]:
provider_data = df_provider[["provider_id"]]

<IPython.core.display.Javascript object>

In [76]:
provider_data

,provider_id
0,1000047
1,1000033
2,1000061
3,1000102
4,1000094
...,...
102,1000071
103,1000034
104,1000079
105,1000005


<IPython.core.display.Javascript object>

In [77]:
provider_data["merchant_name"] = df_provider[["restaurant_name"]]

<IPython.core.display.Javascript object>

In [78]:
df.head(3)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id,price_min,price_max
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002540,1000047,1.59,3.69
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's,https://cn-geo1.uber.com/image-proc/resize/eat...,american,hamilton,1002540,1000047,1.59,3.69
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bacon,hamilton,1002540,1000047,1.59,3.69


<IPython.core.display.Javascript object>

In [79]:
provider_data['address'] = '227'
provider_data['road'] = 'Nguyễn Văn Cừ'
provider_data['ward_id'] = '1'
provider_data['district_id'] = '2'
provider_data['city_id'] = '3'
provider_data['description'] = df_provider.restaurant_description
provider_data['keyword'] = 'abc'
provider_data['hotline'] = '0123456789'
provider_data['tax_code'] = '9876543210'
provider_data['price_range'] = str(20-150)
provider_data['avatar'] = df_provider.avt_restaurant
provider_data['cover_picture'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSDbdE3wNH9nQqdx7FDXPgKBYDkCka871-Jpw&usqp=CAU'
provider_data['facade_photo'] = 'https://frameru.com/wp-content/uploads/2021/06/Restaurant-facade-logo-mockup.jpg'
provider_data['latitude'] = '10.762328'
provider_data['longitude'] = '106.6805406'
provider_data['estimated_cooking_time'] = '30'
provider_data['status'] = 1
provider_data['erc'] = 'https://media-cdn.tripadvisor.com/media/photo-s/13/e5/43/22/fb-img-1532845074912.jpg'
# provider_data['rush_hour'] = '16h-19h'
provider_data['rush_hour'] = datetime.time(3,45,12)

provider_data['order_totals'] = 500
provider_data['user_id'] = df_user[df_user.role == 2].iloc[:107,:]["user_id"].values
provider_data['owner_id'] = df_owner["owner_id"]
provider_data['current_form'] = 5
provider_data['registered_at'] = '2021-12-14 19:49:49'
provider_data['update_at'] = '2021-12-14 19:49:49'
provider_data['delete_at'] = None



<IPython.core.display.Javascript object>

In [80]:
provider_data["price_range"] = df_provider["price_min"].astype(str)
provider_data["price_range"] = (
    provider_data["price_range"] + "-" + df_provider["price_max"].astype(str)
)

<IPython.core.display.Javascript object>

In [81]:
provider_data.iloc[:, 10:15].head(5)

,tax_code,price_range,avatar,cover_picture,facade_photo
0,9876543210,1.59-3.69,https://cn-geo1.uber.com/image-proc/resize/eat...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://frameru.com/wp-content/uploads/2021/06...
1,9876543210,9.0-16.0,https://cn-geo1.uber.com/image-proc/resize/eat...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://frameru.com/wp-content/uploads/2021/06...
2,9876543210,3.79-14.49,https://cn-geo1.uber.com/image-proc/resize/eat...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://frameru.com/wp-content/uploads/2021/06...
3,9876543210,2.74-9.89,https://cn-geo1.uber.com/image-proc/resize/eat...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://frameru.com/wp-content/uploads/2021/06...
4,9876543210,9.0-18.0,https://cn-geo1.uber.com/image-proc/resize/eat...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://frameru.com/wp-content/uploads/2021/06...


<IPython.core.display.Javascript object>

In [88]:
import requests


<IPython.core.display.Javascript object>

In [90]:
# GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

# params = {
#     'address': 'oshiwara industerial center goregaon west mumbai',
#     'sensor': 'false',
#     'region': 'india'
# }

# # Do the request and get the response data
# req = requests.get(GOOGLE_MAPS_API_URL, params=params)
# res = req.json()

# # Use the first result
# result = res['results'][0]

# geodata = dict()
# geodata['lat'] = result['geometry']['location']['lat']
# geodata['lng'] = result['geometry']['location']['lng']
# geodata['address'] = result['formatted_address']

# print('{address}. (lat, lng) = ({lat}, {lng})'.format(**geodata))

<IPython.core.display.Javascript object>

In [64]:
d1 = datetime.datetime.strptime("1/1/2018 1:30 PM", "%m/%d/%Y %I:%M %p")
d2 = datetime.datetime.strptime("1/1/2022 4:50 AM", "%m/%d/%Y %I:%M %p")

# random_date(start, end)

<IPython.core.display.Javascript object>

In [65]:
provider_data["registered_at"] = provider_data["registered_at"].apply(
    lambda x: random_date(d1, d2)
)

<IPython.core.display.Javascript object>

In [66]:
# provider_data["registered_at"] = pd.to_datetime(df_user["registered_at"])
provider_data["update_at"] = provider_data["registered_at"]  # trick nha
# provider_data["delete_at"] = datetime.time(0, 0)
provider_data["delete_at"] = provider_data["registered_at"]

<IPython.core.display.Javascript object>

In [67]:
provider_data.head()

,provider_id,merchant_name,address,road,ward_id,district_id,city_id,description,keyword,hotline,...,status,erc,rush_hour,order_totals,user_id,owner_id,current_form,registered_at,update_at,delete_at
0,1000047,McDonald's (675 Upper James Street),227,Nguyễn Văn Cừ,1,2,3,NaN,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000002,1000000,5,2021-02-18 20:22:11,2021-02-18 20:22:11,2021-02-18 20:22:11
1,1000033,HAMBRGR Downtown,227,Nguyễn Văn Cừ,1,2,3,You've found one of the hottest delivery spots...,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000004,1000001,5,2021-06-04 03:09:21,2021-06-04 03:09:21,2021-06-04 03:09:21
2,1000061,Popeyes Louisiana Kitchen (11 King St E),227,Nguyễn Văn Cừ,1,2,3,NaN,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000015,1000002,5,2018-09-25 20:22:28,2018-09-25 20:22:28,2018-09-25 20:22:28
3,1000103,Wendy's (244 Hwy 20 South),227,Nguyễn Văn Cừ,1,2,3,Wish you could ask someone what's best here? O...,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000017,1000003,5,2020-11-25 15:26:09,2020-11-25 15:26:09,2020-11-25 15:26:09
4,1000094,The Ship,227,Nguyễn Văn Cừ,1,2,3,NaN,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000018,1000004,5,2019-09-20 15:13:28,2019-09-20 15:13:28,2019-09-20 15:13:28


<IPython.core.display.Javascript object>

In [68]:
provider_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   provider_id             108 non-null    int64         
 1   merchant_name           108 non-null    object        
 2   address                 108 non-null    object        
 3   road                    108 non-null    object        
 4   ward_id                 108 non-null    object        
 5   district_id             108 non-null    object        
 6   city_id                 108 non-null    object        
 7   description             53 non-null     object        
 8   keyword                 108 non-null    object        
 9   hotline                 108 non-null    object        
 10  tax_code                108 non-null    object        
 11  price_range             108 non-null    object        
 12  avatar                  108 non-null    object    

<IPython.core.display.Javascript object>

In [69]:
provider_data.description = provider_data.description.fillna(np.nan).replace(
    [np.nan], [None]
)

<IPython.core.display.Javascript object>

In [70]:
provider_data.description = provider_data.description.apply(
    lambda x: x.rpartition(".")[0] + "." if x != None else x
)

<IPython.core.display.Javascript object>

In [71]:
# provider_data.description = provider_data.description.apply(
#     lambda x: "?" if x == None else x
# )

<IPython.core.display.Javascript object>

In [72]:
# provider_data.description = "abc"

<IPython.core.display.Javascript object>

In [73]:
# provider_data = provider_data.dropna()

<IPython.core.display.Javascript object>

In [74]:
provider_data.columns

Index(['provider_id', 'merchant_name', 'address', 'road', 'ward_id',
       'district_id', 'city_id', 'description', 'keyword', 'hotline',
       'tax_code', 'price_range', 'avatar', 'cover_picture', 'facade_photo',
       'latitude', 'longitude', 'estimated_cooking_time', 'status', 'erc',
       'rush_hour', 'order_totals', 'user_id', 'owner_id', 'current_form',
       'registered_at', 'update_at', 'delete_at'],
      dtype='object')

<IPython.core.display.Javascript object>

In [75]:
provider_data["delete_at"] = None

<IPython.core.display.Javascript object>

In [1101]:
# conn, cur = connect_db()
# cols = "`,`".join([str(i) for i in list(provider_data.columns)])
# sql = (
#     "INSERT INTO `provider` (`"
#     + cols
#     + "`) VALUES ("
#     + "%s," * (provider_data.shape[1] - 1)
#     + "%s)"
# )

# try:
#     cur.executemany(sql, provider_data.values.tolist())  # owner_values
#     conn.commit()
# except Error as error:
#     # conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
# finally:
#     cur.close()
#     conn.close()

<IPython.core.display.Javascript object>

In [142]:
provider_data.head(2)

,provider_id,merchant_name,address,road,ward_id,district_id,city_id,description,keyword,hotline,...,status,erc,rush_hour,order_totals,user_id,owner_id,current_form,registered_at,update_at,delete_at
0,1000047,McDonald's (675 Upper James Street),227,Nguyễn Văn Cừ,1,2,3,None,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000002,1000000,5,2020-09-04 21:04:21,2020-09-04 21:04:21,None
1,1000033,HAMBRGR Downtown,227,Nguyễn Văn Cừ,1,2,3,You've found one of the hottest delivery spots...,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000004,1000001,5,2021-05-18 19:29:57,2021-05-18 19:29:57,None


In [250]:
provider_data.to_csv('mysql_data/provider.csv', index = False)

In [1103]:
insert_execute_many(provider_data, "Provider")

<IPython.core.display.Javascript object>

### Provider Operation

In [76]:
df_operation_temp = provider_data[['provider_id']]

<IPython.core.display.Javascript object>

In [77]:
df_operation_temp.shape

(108, 1)

<IPython.core.display.Javascript object>

In [78]:
df_operation_sunday = df_operation_temp.copy()
df_operation_monday = df_operation_temp.copy()
df_operation_tuesday = df_operation_temp.copy()
df_operation_wednesday = df_operation_temp.copy()
df_operation_thursday = df_operation_temp.copy()
df_operation_friday = df_operation_temp.copy()
df_operation_saturday = df_operation_temp.copy()
df_operation_sunday  = df_operation_temp.copy()

df_operation_sunday['day'] = 1
df_operation_monday['day'] = 2
df_operation_tuesday['day'] = 3
df_operation_wednesday['day'] = 4
df_operation_thursday['day'] = 5
df_operation_friday['day'] = 6
df_operation_saturday['day'] = 7


<IPython.core.display.Javascript object>

In [79]:
df_operation_sunday.head()

,provider_id,day
0,1000047,1
1,1000033,1
2,1000061,1
3,1000103,1
4,1000094,1


<IPython.core.display.Javascript object>

In [80]:
df_operation_saturday.tail()


,provider_id,day
103,1000034,7
104,1000099,7
105,1000079,7
106,1000005,7
107,1000035,7


<IPython.core.display.Javascript object>

In [81]:
from random import randrange
import datetime 

def create_end_time(start_time, operating_time):
    res = start_time.hour + operating_time
    return 23 if res > 23 else res

<IPython.core.display.Javascript object>

In [82]:
df_operation_sunday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)
df_operation_monday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)
df_operation_tuesday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)
df_operation_wednesday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)
df_operation_thursday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)
df_operation_friday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)
df_operation_saturday['open_time'] = datetime.time(random.randint(0, 23), 0, 0)



df_operation_sunday['open_time'] = df_operation_sunday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0))  
df_operation_monday['open_time'] = df_operation_monday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0)) 
df_operation_tuesday['open_time'] = df_operation_tuesday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0))
df_operation_wednesday['open_time'] = df_operation_wednesday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0))
df_operation_thursday['open_time'] = df_operation_thursday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0))
df_operation_friday['open_time'] = df_operation_friday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0))
df_operation_saturday['open_time'] = df_operation_saturday['open_time'].apply(lambda x: datetime.time(random.randint(0, 13), 0, 0))


<IPython.core.display.Javascript object>

In [83]:
df_operation_friday.head()

,provider_id,day,open_time
0,1000047,6,03:00:00
1,1000033,6,07:00:00
2,1000061,6,11:00:00
3,1000103,6,11:00:00
4,1000094,6,11:00:00


<IPython.core.display.Javascript object>

In [84]:
# df_operation_sunday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)
# df_operation_monday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)
# df_operation_tuesday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)
# df_operation_wednesday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)
# df_operation_thursday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)
# df_operation_friday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)
# df_operation_sunday['close_time'] = datetime.time(random.randint(0, 23), 0, 0)


df_operation_sunday['close_time'] = df_operation_sunday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))  
df_operation_monday['close_time'] = df_operation_monday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))
df_operation_tuesday['close_time']= df_operation_tuesday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))
df_operation_wednesday['close_time'] = df_operation_wednesday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))
df_operation_thursday['close_time'] = df_operation_thursday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))
df_operation_friday['close_time'] = df_operation_friday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))
df_operation_saturday['close_time'] = df_operation_saturday.open_time.apply(lambda x: datetime.time(create_end_time(x, random.randint(12, 20)), 0, 0))

<IPython.core.display.Javascript object>

In [85]:
df_operation_friday.head()

,provider_id,day,open_time,close_time
0,1000047,6,03:00:00,17:00:00
1,1000033,6,07:00:00,20:00:00
2,1000061,6,11:00:00,23:00:00
3,1000103,6,11:00:00,23:00:00
4,1000094,6,11:00:00,23:00:00


<IPython.core.display.Javascript object>

In [86]:
d1 = datetime.datetime.strptime("1/1/2018 1:30 PM", "%m/%d/%Y %I:%M %p")
d2 = datetime.datetime.strptime("1/1/2022 4:50 AM", "%m/%d/%Y %I:%M %p")

<IPython.core.display.Javascript object>

In [87]:
d1

datetime.datetime(2018, 1, 1, 13, 30)

<IPython.core.display.Javascript object>

In [88]:
df_operation_sunday['create_at'] = d1
df_operation_monday['create_at'] = d1
df_operation_tuesday['create_at'] = d1
df_operation_wednesday['create_at'] = d1
df_operation_thursday['create_at'] = d1
df_operation_friday['create_at'] = d1
df_operation_saturday['create_at'] = d1




df_operation_sunday["create_at"] = df_operation_sunday["create_at"].apply(
    lambda x: random_date(d1, d2)
)
df_operation_monday["create_at"] = df_operation_monday["create_at"].apply(
    lambda x: random_date(d1, d2)
)
df_operation_tuesday["create_at"] = df_operation_tuesday["create_at"].apply(
    lambda x: random_date(d1, d2)
)
df_operation_wednesday["create_at"] = df_operation_wednesday["create_at"].apply(
    lambda x: random_date(d1, d2)
)
df_operation_thursday["create_at"] = df_operation_thursday["create_at"].apply(
    lambda x: random_date(d1, d2)
)
df_operation_friday["create_at"] = df_operation_friday["create_at"].apply(
    lambda x: random_date(d1, d2)
)
df_operation_saturday["create_at"] = df_operation_saturday["create_at"].apply(
    lambda x: random_date(d1, d2)
)

<IPython.core.display.Javascript object>

In [89]:
df_operation = pd.concat([df_operation_sunday, df_operation_monday, df_operation_tuesday, 
df_operation_wednesday, df_operation_thursday, df_operation_friday, df_operation_saturday], ignore_index=True)

<IPython.core.display.Javascript object>

In [90]:
df_operation.head()
#df_operation.tail()

,provider_id,day,open_time,close_time,create_at
0,1000047,1,03:00:00,21:00:00,2020-08-12 06:07:04
1,1000033,1,05:00:00,20:00:00,2021-10-12 09:09:41
2,1000061,1,00:00:00,17:00:00,2020-07-28 07:51:35
3,1000103,1,07:00:00,19:00:00,2021-01-21 02:17:20
4,1000094,1,04:00:00,19:00:00,2021-01-29 10:17:29


<IPython.core.display.Javascript object>

In [91]:
df_operation.day.unique()

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

<IPython.core.display.Javascript object>

In [92]:
df_operation.shape

(756, 5)

<IPython.core.display.Javascript object>

In [93]:
df_operation['update_at'] = df_operation.create_at
df_operation['delete_at'] = None

<IPython.core.display.Javascript object>

In [1122]:
df_operation.head()
df_operation.to_csv('provider_operation.csv', index=False)

<IPython.core.display.Javascript object>

In [251]:
df_operation.to_csv('mysql_data/provider_operation.csv', index = False)

In [1123]:
insert_execute_many(df_operation, 'operation')

<IPython.core.display.Javascript object>

### Product

In [94]:
df.head()

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002564,1000047
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's,https://cn-geo1.uber.com/image-proc/resize/eat...,american,hamilton,1002564,1000047
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bacon,hamilton,1002564,1000047
3,NaN,4.3,28,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bakery,hamilton,1002564,1000047
4,NaN,4.4,63,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bar-food,hamilton,1002564,1000047


<IPython.core.display.Javascript object>

In [95]:
provider_data["merchant_name"].is_unique

False

<IPython.core.display.Javascript object>

In [96]:
provider_data.head(1)

,provider_id,merchant_name,address,road,ward_id,district_id,city_id,description,keyword,hotline,...,status,erc,rush_hour,order_totals,user_id,owner_id,current_form,registered_at,update_at,delete_at
0,1000047,McDonald's (675 Upper James Street),227,Nguyễn Văn Cừ,1,2,3,None,abc,0123456789,...,1,https://media-cdn.tripadvisor.com/media/photo-...,03:45:12,500,1000002,1000000,5,2021-02-18 20:22:11,2021-02-18 20:22:11,None


<IPython.core.display.Javascript object>

In [97]:
df.head(1)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002564,1000047


<IPython.core.display.Javascript object>

In [98]:
# df_product = df[["product_id", "provider_id", "product_name"]]

df_product = df[["product_id", "provider_id", "product_name"]]

<IPython.core.display.Javascript object>

In [99]:
df.product_id.max()

1011571

<IPython.core.display.Javascript object>

In [100]:
df_product["product_status"] = 1

df_product["description"] = (
    df["product_description"].fillna(np.nan).replace([np.nan], [None])
)
df_product["description"] = df_product["description"].apply(
    lambda x: x.rpartition(".")[0] + "." if x != None else x
)

df_product["price"] = df.price
df_product["product_image"] = df.product_image
df_product["quantity"] = 1000
df_product["sold_quantity"] = 1000
df_product["position"] = 1
df_product["rating"] = df.rating
df_product["create_at"] = "2021-12-14 19:49:49"
df_product["update_at"] = "2021-12-14 19:49:49"
df_product["delete_at"] = None
df_product["calo"] = 300

<IPython.core.display.Javascript object>

In [101]:
import datetime

<IPython.core.display.Javascript object>

In [102]:
d1 = datetime.datetime.strptime("1/1/2018 1:30 PM", "%m/%d/%Y %I:%M %p")
d2 = datetime.datetime.strptime("1/1/2022 4:50 AM", "%m/%d/%Y %I:%M %p")
df_product["create_at"] = df_product["create_at"].apply(lambda x: random_date(d1, d2))
# provider_data["registered_at"] = pd.to_datetime(df_user["registered_at"])
df_product["update_at"] = df_product["create_at"]  # trick nha
# provider_data["delete_at"] = datetime.time(0, 0)
df_product["delete_at"] = None

<IPython.core.display.Javascript object>

In [103]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511450 entries, 0 to 511449
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   product_id      511450 non-null  int64         
 1   provider_id     511450 non-null  int64         
 2   product_name    509449 non-null  object        
 3   product_status  511450 non-null  int64         
 4   description     286605 non-null  object        
 5   price           504983 non-null  float64       
 6   product_image   253880 non-null  object        
 7   quantity        511450 non-null  int64         
 8   sold_quantity   511450 non-null  int64         
 9   position        511450 non-null  int64         
 10  rating          511450 non-null  float64       
 11  create_at       511450 non-null  datetime64[ns]
 12  update_at       511450 non-null  datetime64[ns]
 13  delete_at       0 non-null       object        
 14  calo            511450 non-null  int

<IPython.core.display.Javascript object>

In [104]:
df_product["price"] = df_product["price"].fillna(0)


<IPython.core.display.Javascript object>

In [105]:
df_product.dropna(subset=["product_name"], inplace=True)

<IPython.core.display.Javascript object>

In [106]:
df["product_image"] = df["product_image"].fillna(
    "tophinhanhdep.com/wp-content/uploads/2021/10/Pure-White-Wallpapers.jpg"
)

<IPython.core.display.Javascript object>

In [107]:
# conn, cur = connect_db()
# cols = "`,`".join([str(i) for i in list(df_product.columns)])
# sql = (
#     "INSERT INTO `product` (`"
#     + cols
#     + "`) VALUES ("
#     + "%s," * (df_product.shape[1] - 1)
#     + "%s)"
# )

# try:
#     cur.executemany(sql, df_product.values.tolist()[50000:60000])  # owner_values
#     conn.commit()
# except Error as error:
#     # conn.rollback()
#     print("Failed to insert into MySQL table {}".format(error))
# finally:
#     cur.close()
#     conn.close()

<IPython.core.display.Javascript object>

In [108]:
df_product.head(5)

,product_id,provider_id,product_name,product_status,description,price,product_image,quantity,sold_quantity,position,rating,create_at,update_at,delete_at,calo
0,1002564,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,3.6,2021-03-07 08:23:48,2021-03-07 08:23:48,None,300
1,1002564,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,4.4,2019-05-10 13:18:04,2019-05-10 13:18:04,None,300
2,1002564,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,3.9,2021-06-21 05:19:37,2021-06-21 05:19:37,None,300
3,1002564,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,4.3,2020-11-20 12:42:03,2020-11-20 12:42:03,None,300
4,1002564,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,4.4,2018-04-03 07:14:43,2018-04-03 07:14:43,None,300


<IPython.core.display.Javascript object>

In [109]:
len(df_product)

509449

<IPython.core.display.Javascript object>

#### drop duplicate product 

In [110]:
df_product_temp = df_product.drop_duplicates(subset=['product_name', 'product_image'])
df_product_temp = df_product_temp.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [111]:
df_product_temp.shape

(12921, 15)

<IPython.core.display.Javascript object>

In [112]:
df_product_temp.head()

,product_id,provider_id,product_name,product_status,description,price,product_image,quantity,sold_quantity,position,rating,create_at,update_at,delete_at,calo
0,1002564,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,3.6,2021-03-07 08:23:48,2021-03-07 08:23:48,None,300
1,1003816,1000047,Egg McMuffin Extra Value Meal [450.0 Cals],1,Egg McMuffin Extra Value Meal [450.,7.29,https://d1ralsognjng37.cloudfront.net/9813c0cf...,1000,1000,1,4.5,2018-07-07 10:29:07,2018-07-07 10:29:07,None,300
2,1001234,1000047,Bacon 'N Egg McMuffin Extra Value Meal [470.0 ...,1,Bacon 'N Egg McMuffin Extra Value Meal [470.,7.29,https://d1ralsognjng37.cloudfront.net/9038ffb5...,1000,1000,1,3.8,2018-11-17 16:21:00,2018-11-17 16:21:00,None,300
3,1009201,1000047,Sausage 'N Egg McMuffin Extra Value Meal [590....,1,Sausage 'N Egg McMuffin Extra Value Meal [590.,7.29,https://d1ralsognjng37.cloudfront.net/c8739211...,1000,1000,1,3.9,2018-09-12 12:14:56,2018-09-12 12:14:56,None,300
4,1006637,1000047,Med Premium Roast Coffee [4.0 Cals],1,Med Premium Roast Coffee [4.,1.99,https://d1ralsognjng37.cloudfront.net/f82f9ca7...,1000,1000,1,3.6,2020-01-19 17:06:13,2020-01-19 17:06:13,None,300


<IPython.core.display.Javascript object>

In [113]:
df_product_temp.product_id = list(range(1000000, len(df_product_temp)+1000000, 1))

<IPython.core.display.Javascript object>

In [114]:
df_product_temp.head()

,product_id,provider_id,product_name,product_status,description,price,product_image,quantity,sold_quantity,position,rating,create_at,update_at,delete_at,calo
0,1000000,1000047,Chicken McMuffin Extra Value Meal [523.0 Cals],1,Chicken McMuffin Extra Value Meal [523.,5.69,https://d1ralsognjng37.cloudfront.net/81325489...,1000,1000,1,3.6,2021-03-07 08:23:48,2021-03-07 08:23:48,None,300
1,1000001,1000047,Egg McMuffin Extra Value Meal [450.0 Cals],1,Egg McMuffin Extra Value Meal [450.,7.29,https://d1ralsognjng37.cloudfront.net/9813c0cf...,1000,1000,1,4.5,2018-07-07 10:29:07,2018-07-07 10:29:07,None,300
2,1000002,1000047,Bacon 'N Egg McMuffin Extra Value Meal [470.0 ...,1,Bacon 'N Egg McMuffin Extra Value Meal [470.,7.29,https://d1ralsognjng37.cloudfront.net/9038ffb5...,1000,1000,1,3.8,2018-11-17 16:21:00,2018-11-17 16:21:00,None,300
3,1000003,1000047,Sausage 'N Egg McMuffin Extra Value Meal [590....,1,Sausage 'N Egg McMuffin Extra Value Meal [590.,7.29,https://d1ralsognjng37.cloudfront.net/c8739211...,1000,1000,1,3.9,2018-09-12 12:14:56,2018-09-12 12:14:56,None,300
4,1000004,1000047,Med Premium Roast Coffee [4.0 Cals],1,Med Premium Roast Coffee [4.,1.99,https://d1ralsognjng37.cloudfront.net/f82f9ca7...,1000,1000,1,3.6,2020-01-19 17:06:13,2020-01-19 17:06:13,None,300


<IPython.core.display.Javascript object>

In [115]:
df_product_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12921 entries, 0 to 12920
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   product_id      12921 non-null  int64         
 1   provider_id     12921 non-null  int64         
 2   product_name    12921 non-null  object        
 3   product_status  12921 non-null  int64         
 4   description     6992 non-null   object        
 5   price           12921 non-null  float64       
 6   product_image   6265 non-null   object        
 7   quantity        12921 non-null  int64         
 8   sold_quantity   12921 non-null  int64         
 9   position        12921 non-null  int64         
 10  rating          12921 non-null  float64       
 11  create_at       12921 non-null  datetime64[ns]
 12  update_at       12921 non-null  datetime64[ns]
 13  delete_at       0 non-null      object        
 14  calo            12921 non-null  int64         
dtypes:

<IPython.core.display.Javascript object>

In [116]:
df_product_temp.delete_at = None


<IPython.core.display.Javascript object>

In [117]:
df_product_temp.description = df_product_temp.description.fillna(np.nan).replace(
    [np.nan], [None]
)

df_product_temp.product_image = df_product_temp.product_image.fillna(np.nan).replace(
    [np.nan], [None]
)

<IPython.core.display.Javascript object>

In [118]:
df_product_temp.to_csv('mysql_data/product.csv', index = False)

<IPython.core.display.Javascript object>

In [1148]:
#insert_execute_many_large_dataset(df_product, "Product")
insert_execute_many_large_dataset(df_product_temp, "Product")

<IPython.core.display.Javascript object>

In [1149]:
# cur.close()
# conn.close()


<IPython.core.display.Javascript object>

###  Category Of Provider

#### Provider Category Detail

In [119]:
df.head(3)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002564,1000047
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's,https://cn-geo1.uber.com/image-proc/resize/eat...,american,hamilton,1002564,1000047
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bacon,hamilton,1002564,1000047


<IPython.core.display.Javascript object>

In [120]:
le = preprocessing.LabelEncoder()
df['restaurant_category_id'] = le.fit_transform(df.restaurant_category)

df['restaurant_category_id'] = df['restaurant_category_id'].apply(lambda x: x+ 1000000)

<IPython.core.display.Javascript object>

In [121]:
df.head(3)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id,restaurant_category_id
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002564,1000047,1000000
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's,https://cn-geo1.uber.com/image-proc/resize/eat...,american,hamilton,1002564,1000047,1000006
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bacon,hamilton,1002564,1000047,1000011


<IPython.core.display.Javascript object>

#### provider_category

In [122]:
df_provider_category = df[
    ["restaurant_category_id", "restaurant_category"]
].drop_duplicates(ignore_index=True)

df_provider_category = df_provider_category.rename(
    columns={
        "restaurant_category_id": "provider_category_id",
        "restaurant_category": "provider_category_name",
    }
)


<IPython.core.display.Javascript object>

In [123]:
df_provider_category.head()

,provider_category_id,provider_category_name
0,1000000,24-hours
1,1000006,american
2,1000011,bacon
3,1000012,bakery
4,1000013,bar-food


<IPython.core.display.Javascript object>

In [124]:
df_provider_category.to_csv('mysql_data/provider_category.csv', index = False)

<IPython.core.display.Javascript object>

In [1155]:
insert_execute_many(df_provider_category, 'providercategory')

<IPython.core.display.Javascript object>

#### provider_category_detail

In [125]:
df_provider_category_detail = df[['restaurant_category_id', 'provider_id']].drop_duplicates(ignore_index=True)

df_provider_category_detail= df_provider_category_detail.rename(
    columns={
        "restaurant_category_id": "provider_category_id",
    }
)

df_provider_category_detail.head()

,provider_category_id,provider_id
0,1000000,1000047
1,1000006,1000047
2,1000011,1000047
3,1000012,1000047
4,1000013,1000047


<IPython.core.display.Javascript object>

In [126]:
df_provider_category_detail.to_csv('mysql_data/provider_category_detail.csv', index = False)

<IPython.core.display.Javascript object>

In [1157]:
insert_execute_many_large_dataset(df_provider_category_detail, 'providercategorydetail')

<IPython.core.display.Javascript object>

#### cuisine_category

In [127]:
cuisine_category = ['afghan', 'african', 'american', 'arabian', 'asian', 'asian-fusion', 'belgian', 'british', 'canadian', 'caribbean', 'chinese', 'colombian', 'european', 'korean', 'dutch', 'eastern-european', 'ethiopian', 'filipino', 'german', 'greek', 'indonesian', 'italian', 'jamaican', 'japanese', 'indian', 'lebanese', 'latin-american', 'english', 'malaysian', 'mediterranean', 'mexican',  'french', 'modern-asian', 'modern-european', 'new-american', 'new-canadian', 'new-mexican',  'middle-eastern',  'nepalese', 'north-indian', 'northern-thai',  'persian', 'portuguese', 'halal', 'shanghai', 'south-american', 'south-asian', 'south-east-asian', 'south-indian', 'southern', 'southern-thai', 'taiwanese', 'thai', 'traditional-american', 'vietnamese', 'western', 'latin-fusion', 'spanish', 'others']

<IPython.core.display.Javascript object>

In [128]:
df['cuisine_category_name'] = df.restaurant_category.apply(lambda x: 'others' if x not in cuisine_category else x)

<IPython.core.display.Javascript object>

In [129]:
df.head(3)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id,restaurant_category_id,cuisine_category_name
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002564,1000047,1000000,others
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's,https://cn-geo1.uber.com/image-proc/resize/eat...,american,hamilton,1002564,1000047,1000006,american
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bacon,hamilton,1002564,1000047,1000011,others


<IPython.core.display.Javascript object>

In [130]:
df['cuisine_category_id'] = le.fit_transform(df.cuisine_category_name)
df['cuisine_category_id'] = df['cuisine_category_id'].apply(lambda x: x + 1000000)

<IPython.core.display.Javascript object>

In [131]:
df_cuisine = df[['cuisine_category_id', 'cuisine_category_name']].drop_duplicates(ignore_index=True) 

<IPython.core.display.Javascript object>

In [132]:
df_cuisine.head()

,cuisine_category_id,cuisine_category_name
0,1000032,others
1,1000002,american
2,1000041,western
3,1000031,northern-thai
4,1000038,southern-thai


<IPython.core.display.Javascript object>

In [133]:
df_cuisine.to_csv('mysql_data/cuisine_category.csv', index = False)

<IPython.core.display.Javascript object>

In [1164]:
insert_execute_many(df_cuisine, "CuisineCategory")

<IPython.core.display.Javascript object>

#### cuisine_category_detail

In [134]:
df_cuisine_category_detail = df[['cuisine_category_id', 'provider_id']].drop_duplicates(ignore_index=True)
df_cuisine_category_detail.head()

,cuisine_category_id,provider_id
0,1000032,1000047
1,1000002,1000047
2,1000041,1000047
3,1000032,1000033
4,1000002,1000033


<IPython.core.display.Javascript object>

In [135]:
df_cuisine_category_detail.to_csv('mysql_data/cuisine_category_detail.csv', index = False)

<IPython.core.display.Javascript object>

In [1166]:
insert_execute_many_large_dataset(df_cuisine_category_detail, "CuisineCategoryDetail")

<IPython.core.display.Javascript object>

#### MainFoodCategory - Food Category

In [136]:
main_food_category = {
    "healthy": [
        "acai",
        "allergy-friendly",
        "gluten-free",
        "kids-friendly",
        "healthy",
        "low-calorie",
        "low-carb",
        "organic",
    ],
    "daily_meal": [
        "appetizers",
        "dessert",
        "late-night",
        "bread",
        "soup",
        "mac-and-cheese",
        "noodles",
        "salad",
    ],
    "vegan": ["vegan-friendly", "vegetarian", "vegetarian-friendly"],
    "fast_food": ["bacon", "burger", "fast-food", "frozen-food", "sandwich", "wings"],
    "beverage": [
        "drink",
        "juice-and-smoothies",
        "milkshake",
    ],
    "alcool": ["wine", "rum"],
    "junk_food": ["ice-cream-and-frozen-yogurt", "matcha", "tapas", "snacks"],
    "proteins": ["beef", "cured-meat", "eggs", "lobster", "oysters", "seafood", "ribs"],
    "sweets": [
        "cupcake",
        "cake",
        "candy",
        "custard",
        "donut",
        "sweets",
        "pastry",
        "pie",
    ],
    "specialty": [
        "bento",
        "biryani",
        "charcuterie",
        "congee",
        "couscous",
        "curry",
        "dim-sum",
        "dosa",
        "fish-and-chips",
        "indian-curry",
        "japanese-curry",
        "kebabs",
        "korean-bbq",
        "lasagne",
        "pho",
        "pizza",
        "pasta",
        "yakitori",
        "tex-mex",
        "ramen",
        "paella",
        "poke",
        "rolls",
        "shawarma",
        "udon",
        "pollo",
        "poutine",
        "pretzel",
        "soul-food",
    ],
    "raw": ["raw", "sushi", "sashimi", "salmon"],
    "others": ["bar-food", "bbq", "grill", "hotpot", "chili", "spirits", "wrap"],
}

<IPython.core.display.Javascript object>

In [137]:
food_category = ['acai', 'allergy-friendly', 'appetizers', 'bacon', 'bar-food', 'bbq', 'beef', 'bento', 'biryani', 'burger', 'charcuterie', 'chili', 'congee', 'cupcake', 'bread', 'cake', 'candy', 'cured-meat', 'couscous', 'curry', 'custard',  'dessert', 'dim-sum',  'donut', 'dosa', 'drink', 'eggs', 'fast-food', 'fish-and-chips', 'frozen-food', 'gluten-free', 'grill', 'hotpot',   'ice-cream-and-frozen-yogurt', 'indian-curry', 'japanese-curry', 'juice-and-smoothies', 'kebabs', 'kids-friendly', 'healthy', 'korean-bbq', 'lasagne',   'late-night', 'soup', 'milkshake',   'lobster', 'low-calorie', 'low-carb', 'mac-and-cheese', 'matcha', 'oysters', 'pho', 'pizza', 'wine', 'sushi',  'rum', 'pasta',  'tapas', 'tex-mex', 'noodles', 'organic', 'paella', 'pie',  'poke', 'ramen', 'raw', 'ribs', 'rolls','salad', 'salmon', 'snacks', 'sandwich', 'sashimi', 'seafood', 'shawarma', 'vegan-friendly', 'vegetarian', 'udon', 'wings',  'wrap', 'yakitori',  'spirits', 'sweets', 'pastry',  'vegetarian-friendly', 'pollo', 'poutine', 'pretzel', 'soul-food']


<IPython.core.display.Javascript object>

In [138]:
df['food_category_name'] = df.restaurant_category.apply(lambda x: 'others' if x not in food_category else x)

<IPython.core.display.Javascript object>

In [139]:
df.head(5)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,restaurant_name,avt_restaurant,restaurant_category,city,product_id,provider_id,restaurant_category_id,cuisine_category_name,cuisine_category_id,food_category_name
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,24-hours,hamilton,1002564,1000047,1000000,others,1000032,others
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's,https://cn-geo1.uber.com/image-proc/resize/eat...,american,hamilton,1002564,1000047,1000006,american,1000002,others
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bacon,hamilton,1002564,1000047,1000011,others,1000032,bacon
3,NaN,4.3,28,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bakery,hamilton,1002564,1000047,1000012,others,1000032,others
4,NaN,4.4,63,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,McDonald's (675 Upper James Street),https://cn-geo1.uber.com/image-proc/resize/eat...,bar-food,hamilton,1002564,1000047,1000013,others,1000032,bar-food


<IPython.core.display.Javascript object>

In [140]:
df['food_category_id'] = le.fit_transform(df['food_category_name'])
df['food_category_id'] = df['food_category_id'].apply(lambda x: x + 1000000)

<IPython.core.display.Javascript object>

In [141]:
list_main = list(main_food_category.keys())
print(list_main)

['healthy', 'daily_meal', 'vegan', 'fast_food', 'beverage', 'alcool', 'junk_food', 'proteins', 'sweets', 'specialty', 'raw', 'others']


<IPython.core.display.Javascript object>

In [142]:
def map_main_food_id(fied):
    id = -1
    if fied in [
        "acai",
        "allergy-friendly",
        "gluten-free",
        "kids-friendly",
        "healthy",
        "low-calorie",
        "low-carb",
        "organic",
    ]:
        id = 1000000
    elif fied in [
        "appetizers",
        "dessert",
        "late-night",
        "bread",
        "soup",
        "mac-and-cheese",
        "noodles",
        "salad",
    ]:
        id = 1000001
    elif fied in ["vegan-friendly", "vegetarian", "vegetarian-friendly"]:
        id = 1000002
    elif fied in ["bacon", "burger", "fast-food", "frozen-food", "sandwich", "wings"]:
        id = 1000003
    elif fied in [
        "drink",
        "juice-and-smoothies",
        "milkshake",
    ]:
        id = 1000004
    elif fied in ["wine", "rum"]:
        id = 1000005
    elif fied in ["ice-cream-and-frozen-yogurt", "matcha", "tapas", "snacks"]:
        id = 1000006
    elif fied in [
        "beef",
        "cured-meat",
        "eggs",
        "lobster",
        "oysters",
        "seafood",
        "ribs",
    ]:
        id = 1000007
    elif fied in [
        "cupcake",
        "cake",
        "candy",
        "custard",
        "donut",
        "sweets",
        "pastry",
        "pie",
    ]:
        id = 1000008
    elif fied in [
        "bento",
        "biryani",
        "charcuterie",
        "congee",
        "couscous",
        "curry",
        "dim-sum",
        "dosa",
        "fish-and-chips",
        "indian-curry",
        "japanese-curry",
        "kebabs",
        "korean-bbq",
        "lasagne",
        "pho",
        "pizza",
        "pasta",
        "yakitori",
        "tex-mex",
        "ramen",
        "paella",
        "poke",
        "rolls",
        "shawarma",
        "udon",
        "pollo",
        "poutine",
        "pretzel",
        "soul-food",
    ]:
        id = 1000009
    elif fied in ["raw", "sushi", "sashimi", "salmon"]:
        id = 1000010
    else:
        id = 1000011
    return id

<IPython.core.display.Javascript object>

In [143]:
def map_main_food_name(fied):
    name = 'others'
    if fied in [
        "acai",
        "allergy-friendly",
        "gluten-free",
        "kids-friendly",
        "healthy",
        "low-calorie",
        "low-carb",
        "organic",
    ]:
        name = 'healthy'
    elif fied in [
        "appetizers",
        "dessert",
        "late-night",
        "bread",
        "soup",
        "mac-and-cheese",
        "noodles",
        "salad",
    ]:
        name = 'daily_meal'
    elif fied in ["vegan-friendly", "vegetarian", "vegetarian-friendly"]:
        name = 'vegan'
    elif fied in ["bacon", "burger", "fast-food", "frozen-food", "sandwich", "wings"]:
        name = 'fast food'
    elif fied in [
        "drink",
        "juice-and-smoothies",
        "milkshake",
    ]:
        name = 'beverage'
    elif fied in ["wine", "rum"]:
        name = 'alcohol'
    elif fied in ["ice-cream-and-frozen-yogurt", "matcha", "tapas", "snacks"]:
        name = 'junk_food'
    elif fied in [
        "beef",
        "cured-meat",
        "eggs",
        "lobster",
        "oysters",
        "seafood",
        "ribs",
    ]:
        name = 'protein'
    elif fied in [
        "cupcake",
        "cake",
        "candy",
        "custard",
        "donut",
        "sweets",
        "pastry",
        "pie",
    ]:
        name = 'sweet'
    elif fied in [
        "bento",
        "biryani",
        "charcuterie",
        "congee",
        "couscous",
        "curry",
        "dim-sum",
        "dosa",
        "fish-and-chips",
        "indian-curry",
        "japanese-curry",
        "kebabs",
        "korean-bbq",
        "lasagne",
        "pho",
        "pizza",
        "pasta",
        "yakitori",
        "tex-mex",
        "ramen",
        "paella",
        "poke",
        "rolls",
        "shawarma",
        "udon",
        "pollo",
        "poutine",
        "pretzel",
        "soul-food",
    ]:
        name = 'specialty'

    elif fied in ["raw", "sushi", "sashimi", "salmon"]:
        name = 'raw'
    return name

<IPython.core.display.Javascript object>

In [144]:
df['main_food_category_name'] = df.food_category_name.apply(lambda x: map_main_food_name(x))

<IPython.core.display.Javascript object>

In [145]:
df.head(5)

,restaurant_description,rating,num_review,menu_category,product_name,price,product_description,status,product_image,restaurant_url,...,restaurant_category,city,product_id,provider_id,restaurant_category_id,cuisine_category_name,cuisine_category_id,food_category_name,food_category_id,main_food_category_name
0,NaN,3.6,82,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,...,24-hours,hamilton,1002564,1000047,1000000,others,1000032,others,1000051,others
1,NaN,4.4,25,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,...,american,hamilton,1002564,1000047,1000006,american,1000002,others,1000051,others
2,NaN,3.9,124,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,...,bacon,hamilton,1002564,1000047,1000011,others,1000032,bacon,1000003,fast food
3,NaN,4.3,28,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,...,bakery,hamilton,1002564,1000047,1000012,others,1000032,others,1000051,others
4,NaN,4.4,63,Most Popular,Chicken McMuffin Extra Value Meal [523.0 Cals],5.69,Chicken McMuffin Extra Value Meal [523.0 Cals],In stock,https://d1ralsognjng37.cloudfront.net/81325489...,https://www.ubereats.com/ca/store/mcdonalds-67...,...,bar-food,hamilton,1002564,1000047,1000013,others,1000032,bar-food,1000004,others


<IPython.core.display.Javascript object>

In [146]:
df['main_food_category_name'].unique()

array(['others', 'fast food', 'daily_meal', 'sweet', 'beverage',
       'protein', 'specialty', 'junk_food', 'healthy', 'vegan', 'alcohol',
       'raw'], dtype=object)

<IPython.core.display.Javascript object>

In [147]:
df['main_food_category_id'] = le.fit_transform(df['main_food_category_name'])
df['main_food_category_id'] = df['main_food_category_id'].apply(lambda x: x+1000000)

<IPython.core.display.Javascript object>

In [148]:
df_main_food_category = df[['main_food_category_id', 'main_food_category_name']].drop_duplicates(ignore_index=True)

<IPython.core.display.Javascript object>

In [149]:
df_main_food_category.shape

(12, 2)

<IPython.core.display.Javascript object>

In [150]:
df_main_food_category.to_csv('mysql_data/main_food_category.csv', index = False)

<IPython.core.display.Javascript object>

In [1181]:
insert_execute_many(df_main_food_category, "MainFoodCategory")

<IPython.core.display.Javascript object>

#### Main Food Category Detail

In [151]:
# df_main_food_category_detail = df.drop_duplicates(
#     subset=["product_name", "product_image"]
# )[["main_food_category_id", "product_id"]].drop_duplicates(ignore_index=True)


df_main_food_category_detail = df[
    ["main_food_category_id", "product_id"]
].drop_duplicates(ignore_index=True)


<IPython.core.display.Javascript object>

In [152]:
df_main_food_category_detail.main_food_category_id.unique()

array([1000006, 1000003, 1000002, 1000010, 1000001, 1000007, 1000009,
       1000005, 1000004, 1000011, 1000000, 1000008], dtype=int64)

<IPython.core.display.Javascript object>

In [153]:
df_main_food_category_detail.head()

,main_food_category_id,product_id
0,1000006,1002564
1,1000003,1002564
2,1000002,1002564
3,1000010,1002564
4,1000001,1002564


<IPython.core.display.Javascript object>

In [154]:
df_main_food_category_detail.shape

(101764, 2)

<IPython.core.display.Javascript object>

In [155]:
df_main_food_category_detail.to_csv('mysql_data/main_food_category_detail.csv', index = False)

<IPython.core.display.Javascript object>

In [156]:
insert_execute_many_large_dataset(df_main_food_category_detail, 'MainFoodCategoryDetail')

Failed to insert into MySQL table 1062 (23000): Duplicate entry '1002564-1000006' for key 'mainfoodcategorydetail.PRIMARY'
Failed to insert into MySQL table 1062 (23000): Duplicate entry '1005950-1000009' for key 'mainfoodcategorydetail.PRIMARY'


<IPython.core.display.Javascript object>

#### Food Category

In [157]:
df_food_category = df[['food_category_name', 'food_category_id', 'main_food_category_id']].drop_duplicates(ignore_index=True)

<IPython.core.display.Javascript object>

In [158]:
df_food_category.shape

(88, 3)

<IPython.core.display.Javascript object>

In [159]:
df_food_category.to_csv('mysql_data/food_category.csv', index = False)

<IPython.core.display.Javascript object>

In [1190]:
insert_execute_many(df_food_category, 'FoodCategory')

<IPython.core.display.Javascript object>

#### Food Category Detail

In [160]:
df_food_category_detail = df[["food_category_id", "product_id"]].drop_duplicates(
    ignore_index=True
)


<IPython.core.display.Javascript object>

In [161]:
df_food_category_detail.head(5)

,food_category_id,product_id
0,1000051,1002564
1,1000003,1002564
2,1000004,1002564
3,1000010,1002564
4,1000021,1002564


<IPython.core.display.Javascript object>

In [162]:
df_food_category_detail.shape

(276304, 2)

<IPython.core.display.Javascript object>

In [163]:
df_food_category_detail.to_csv('mysql_data/food_category_detail.csv', index = False)

<IPython.core.display.Javascript object>

In [1194]:
insert_execute_many_large_dataset(df_food_category_detail, 'FoodCategoryDetail')

<IPython.core.display.Javascript object>

#### Menu Category

In [233]:
df.menu_category = df.menu_category.fillna('Others')

In [234]:
df['menu_category_id'] = le.fit_transform(df.menu_category)
df['menu_category_id'] = df['menu_category_id'] .apply(lambda x: x+1000000)

In [235]:
df_menu_category =  df[['menu_category', 'provider_id', 'menu_category_id']]
df_menu_category['position'] = 1

In [236]:
df_menu_category

,menu_category,provider_id,menu_category_id,position
0,Most Popular,1000047,1000412,1
1,Most Popular,1000047,1000412,1
2,Most Popular,1000047,1000412,1
3,Most Popular,1000047,1000412,1
4,Most Popular,1000047,1000412,1
...,...,...,...,...
511445,Pet,1000035,1000459,1
511446,Pet,1000035,1000459,1
511447,Pet,1000035,1000459,1
511448,Pet,1000035,1000459,1


In [237]:
df_menu_category = df_menu_category.drop_duplicates(subset=['menu_category', 'provider_id'],ignore_index=True)

In [238]:
df_menu_category

,menu_category,provider_id,menu_category_id,position
0,Most Popular,1000047,1000412,1
1,Breakfast Sandwiches and Burritos,1000047,1000080,1
2,Hot Drinks,1000047,1000333,1
3,Cold Drinks,1000047,1000170,1
4,Snacks & Sides,1000047,1000578,1
...,...,...,...,...
1138,Better for you Beverage,1000035,1000057,1
1139,Cold Coffee & Iced Tea,1000035,1000167,1
1140,Soft Drinks,1000035,1000580,1
1141,Confectionery,1000035,1000180,1


In [239]:
df_menu_category["menu_id"] = list(
    range(1000000, 1000000 + df_menu_category.shape[0], 1)
)


In [240]:
df_menu_category = df_menu_category[['menu_id', 'menu_category', 'provider_id', 'position']]

df_menu_category = df_menu_category.rename(columns={'menu_category': 'name'})

In [241]:
df_menu_category

,menu_id,name,provider_id,position
0,1000000,Most Popular,1000047,1
1,1000001,Breakfast Sandwiches and Burritos,1000047,1
2,1000002,Hot Drinks,1000047,1
3,1000003,Cold Drinks,1000047,1
4,1000004,Snacks & Sides,1000047,1
...,...,...,...,...
1138,1001138,Better for you Beverage,1000035,1
1139,1001139,Cold Coffee & Iced Tea,1000035,1
1140,1001140,Soft Drinks,1000035,1
1141,1001141,Confectionery,1000035,1


In [242]:
df_menu_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   menu_id      1143 non-null   int64 
 1   name         1143 non-null   object
 2   provider_id  1143 non-null   int64 
 3   position     1143 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 35.8+ KB


In [243]:
df_menu_category.to_csv('mysql_data/menu_category.csv', index = False)

In [1205]:
insert_execute_many_large_dataset(df_menu_category, 'MenuCategory')

<IPython.core.display.Javascript object>

#### Menu Category Detail

In [244]:
df_menu_category_detail = df[['menu_category_id', 'product_id']]

In [245]:
df_menu_category_detail = df_menu_category_detail.rename(
    columns={"menu_category_id": "menu_id"}
)


In [246]:
df_menu_category_detail

,menu_id,product_id
0,1000412,1002564
1,1000412,1002564
2,1000412,1002564
3,1000412,1002564
4,1000412,1002564
...,...,...
511445,1000459,1006762
511446,1000459,1006762
511447,1000459,1006762
511448,1000459,1006762


In [247]:
df_menu_category_detail = df_menu_category_detail.drop_duplicates()

In [248]:
df_menu_category_detail.to_csv('mysql_data/menu_category_detail.csv', index = False)

In [1210]:
insert_execute_many_large_dataset(df_menu_category_detail, 'MenuCategoryDetail')

<IPython.core.display.Javascript object>

### TEST
---

### Fix menu category

In [290]:
df_new = pd.read_csv('data/output_restaurant.csv')


<IPython.core.display.Javascript object>

In [291]:
df_new.menu_category = df_new.menu_category.fillna('Others')

<IPython.core.display.Javascript object>

In [292]:
le = preprocessing.LabelEncoder()
df_new["product_id"] = le.fit_transform(df_new['product_name'])

df_new["product_id"] = df_new["product_id"].apply(lambda x: x+1000000)

<IPython.core.display.Javascript object>

In [293]:
df_new["provider_id"] = le.fit_transform(df_new['restaurant_url'])

df_new["provider_id"] = df_new["provider_id"].apply(lambda x: x+1000000)

<IPython.core.display.Javascript object>

In [294]:
df_new.dropna(subset=["product_name"], inplace=True)


<IPython.core.display.Javascript object>

In [295]:
df_new.reset_index(inplace = True)

<IPython.core.display.Javascript object>

In [296]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509449 entries, 0 to 509448
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   509449 non-null  int64  
 1   restaurant_description  290189 non-null  object 
 2   rating                  509449 non-null  float64
 3   num_review              509449 non-null  int64  
 4   menu_category           509449 non-null  object 
 5   product_name            509449 non-null  object 
 6   price                   504914 non-null  float64
 7   product_description     286605 non-null  object 
 8   status                  480711 non-null  object 
 9   product_image           253880 non-null  object 
 10  restaurant_url          509449 non-null  object 
 11  restaurant_name         509449 non-null  object 
 12  avt_restaurant          509201 non-null  object 
 13  restaurant_category     509449 non-null  object 
 14  city                

<IPython.core.display.Javascript object>

In [297]:
df_new_2 = df_new[['menu_category', 'product_id', 'product_name', 'provider_id']]

<IPython.core.display.Javascript object>

In [301]:
df_new_2.shape

(13972, 5)

<IPython.core.display.Javascript object>

In [299]:
df_new_2 = df_new_2.drop_duplicates()

<IPython.core.display.Javascript object>

In [300]:
df_new_2["menu_id"] = list(range(1000000, 1000000 + len(df_new_2), 1))

<IPython.core.display.Javascript object>

In [302]:
df_new_2.head()

,menu_category,product_id,product_name,provider_id,menu_id
0,Most Popular,1002564,Chicken McMuffin Extra Value Meal [523.0 Cals],1000047,1000000
29,Most Popular,1003816,Egg McMuffin Extra Value Meal [450.0 Cals],1000047,1000001
58,Most Popular,1001234,Bacon 'N Egg McMuffin Extra Value Meal [470.0 ...,1000047,1000002
87,Most Popular,1009201,Sausage 'N Egg McMuffin Extra Value Meal [590....,1000047,1000003
116,Breakfast Sandwiches and Burritos,1002564,Chicken McMuffin Extra Value Meal [523.0 Cals],1000047,1000004


<IPython.core.display.Javascript object>

In [264]:
df_menu_category_new =  df_new_2[['menu_id', 'menu_category', 'provider_id']]

<IPython.core.display.Javascript object>

In [265]:
df_menu_category_new = df_menu_category_new.rename(columns={'menu_category':'name'})

<IPython.core.display.Javascript object>

In [266]:
df_menu_category_new

,menu_id,name,provider_id
0,1000000,Most Popular,1000047
29,1000001,Most Popular,1000047
58,1000002,Most Popular,1000047
87,1000003,Most Popular,1000047
116,1000004,Breakfast Sandwiches and Burritos,1000047
...,...,...,...
509424,1013967,Pet,1000035
509429,1013968,Pet,1000035
509434,1013969,Pet,1000035
509439,1013970,Pet,1000035


<IPython.core.display.Javascript object>

In [267]:
df_menu_category_new.shape

(13972, 3)

<IPython.core.display.Javascript object>

In [274]:
df_menu_category_new = df_menu_category_new.drop_duplicates(subset=['name', 'provider_id'])

<IPython.core.display.Javascript object>

In [275]:
df_menu_category_new.shape

(1142, 4)

<IPython.core.display.Javascript object>

In [270]:
df_menu_category_new['position'] = 1

<IPython.core.display.Javascript object>

In [276]:
df_menu_category_new

,menu_id,name,provider_id,position
0,1000000,Most Popular,1000047,1
116,1000004,Breakfast Sandwiches and Burritos,1000047,1
232,1000008,Hot Drinks,1000047,1
377,1000013,Cold Drinks,1000047,1
928,1000032,Snacks & Sides,1000047,1
...,...,...,...,...
508929,1013868,Better for you Beverage,1000035,1
509004,1013883,Cold Coffee & Iced Tea,1000035,1
509124,1013907,Soft Drinks,1000035,1
509194,1013921,Confectionery,1000035,1


<IPython.core.display.Javascript object>

In [277]:
insert_execute_many(df_menu_category_new, 'MenuCategory')

<IPython.core.display.Javascript object>

### Menu category Detail Fix

In [279]:
df_new_menu_detail = df_new_2[['menu_id', 'product_id']]

<IPython.core.display.Javascript object>

In [280]:
df_new_menu_detail

,menu_id,product_id
0,1000000,1002564
29,1000001,1003816
58,1000002,1001234
87,1000003,1009201
116,1000004,1002564
...,...,...
509424,1013967,1004337
509429,1013968,1008466
509434,1013969,1008475
509439,1013970,1007987


<IPython.core.display.Javascript object>

In [281]:
df_new_menu_detail = df_new_menu_detail.drop_duplicates()

<IPython.core.display.Javascript object>

In [282]:
insert_execute_many_large_dataset(df_new_menu_detail, 'MenuCategoryDetail')

Failed to insert into MySQL table 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`tastie`.`menucategorydetail`, CONSTRAINT `FK_MenuCategories_belongto_MenuCategory` FOREIGN KEY (`menu_id`) REFERENCES `menucategory` (`menu_id`))


<IPython.core.display.Javascript object>

In [1211]:
# user_name = "root"
# pwd = "Trung99"
# server = "localhost"
# db = "tastie"

# conn_str = "mysql+pymysql://{user}:{password}@{host}/{database}?charset=utf8".format(
#     user=user_name, password=pwd, host=server, database=db
# )

# %sql $conn_str

<IPython.core.display.Javascript object>

In [1212]:
# df.res

<IPython.core.display.Javascript object>

In [1219]:
df.to_csv('test.csv', index=False)

<IPython.core.display.Javascript object>

In [1213]:
# cur.close()
# conn.close()

<IPython.core.display.Javascript object>

In [1214]:
# from geopy.geocoders import Nominatim

<IPython.core.display.Javascript object>

In [1215]:
# geolocator = Nominatim(user_agent="specify_your_app_name_here")

<IPython.core.display.Javascript object>